In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def check_k_anonymity(df, quasi_identifiers, k):
    # Raggruppare il dataframe per i quasi-identificatori e ottenere la dimensione di ogni gruppo
    group_sizes = df.groupby(quasi_identifiers).size()
    num_partitions = len(group_sizes)
    total_rows = len(df)
    
    # Verificare se tutti i gruppi hanno almeno k elementi e al massimo k*2 elementi
    is_k_anonymous = ((group_sizes >= k)).all()
    #is_k_anonymous = ((group_sizes >= k) & (group_sizes <= k*2)).all()

    
    # Stampa del risultato della verifica di k-anonimity
    if is_k_anonymous:
        print(f"Il dataset rispetta la k-anonimity per k compreso tra {k} e {k*2}")
    else:
        print(f"Il dataset non rispetta la k-anonimity per k compreso tra {k} e {k*2}")
        print("\nLe seguenti partizioni non rispettano la k-anonimity:")
        print(group_sizes[(group_sizes < k) | (group_sizes > k*2)])
    
    return is_k_anonymous

# Carica il dataset
anonymized_df = pd.read_csv('anonymized.csv')
original_df = pd.read_csv('generation/database.csv')   

# Definisci i quasi-identificatori e il valore di k
quasi_identifiers = ['age', 'gender', 'city', 'education', 'profession']
k = 3  # Sostituisci con il valore di k desiderato

# Verifica la k-anonimity
check_k_anonymity(anonymized_df, quasi_identifiers, k)


Il dataset rispetta la k-anonimity per k compreso tra 3 e 6


True

In [2]:
numerical_qis = ['age']
ordinal_qis = ['education', 'gender']
categorical_qis = ['city', 'profession']
all_qis = categorical_qis + numerical_qis + ordinal_qis

Original dataset age average and anonymized dataset age average: 

In [3]:
import pandas as pd
import numpy as np

# Funzione per calcolare la media di un range di età
def calculate_mean(column):
    try:
        # Rimuoviamo parentesi quadre, se presenti
        col_range = column.strip('[]')
        # Dividiamo il range in due numeri
        col_split = col_range.split('-')
        # Calcoliamo la media
        return np.mean([int(col_split[0]), int(col_split[1])])
    except (IndexError, ValueError):
        # Se c'è un errore nell'indicizzazione o nella conversione, restituisci NaN
        return np.nan


# Calcoliamo la media dei range di età in anonymized_df
anonymized_df['age_mean'] = anonymized_df['age'].apply(calculate_mean)

# Calcoliamo la media dell'età in original_df
original_age_mean = original_df['age'].mean()

# Stampa dei risultati
print("Media dei range di età in anonymized_df:")
print(anonymized_df[['age', 'age_mean']])

print("\nMedia dell'età in original_df:")
print(original_age_mean)


Media dei range di età in anonymized_df:
         age  age_mean
0    [20-66]      43.0
1    [20-66]      43.0
2    [20-66]      43.0
3    [22-66]      44.0
4    [22-66]      44.0
..       ...       ...
995  [24-86]      55.0
996  [24-86]      55.0
997  [27-67]      47.0
998  [27-67]      47.0
999  [27-67]      47.0

[1000 rows x 2 columns]

Media dell'età in original_df:
52.984


Original and anonymized modal values of categorical attributes:

In [4]:
import pandas as pd
import numpy as np

# Funzione per rimuovere parentesi quadre
def remove_brackets(value):
    return value.strip('[]')

# Funzione per calcolare la moda degli attributi categorici
def calculate_mode(df, categorical_qis):
    # Controllo per determinare se il DataFrame ha parentesi quadre nei valori
    first_value = df.iloc[0][categorical_qis[0]]
    if isinstance(first_value, str) and '[' in first_value:
        # Se ci sono parentesi quadre, applica la funzione remove_brackets
        for col in categorical_qis:
            df[col] = df[col].apply(remove_brackets)

    # Calcola la moda per le colonne categoriche
    modes = df[categorical_qis].mode().iloc[0]

    return modes

# Esempio di utilizzo con entrambi i DataFrame
if __name__ == "__main__":
    # Carica il DataFrame anonymized_df (sostituire con il caricamento effettivo del file)
    anonymized_df = pd.read_csv('anonymized.csv')

    # Carica il DataFrame original_df (sostituire con il caricamento effettivo del file)
    original_df = pd.read_csv('generation/database.csv')

    # Definisci i quasi identificatori categorici
    categorical_qis = ['gender', 'city', 'age', 'profession', 'education']

    # Calcola la moda degli attributi categorici per anonymized_df
    modes_anonymized = calculate_mode(anonymized_df, categorical_qis)
    print("Moda degli attributi categorici per anonymized_df:")
    print(modes_anonymized)

    # Calcola la moda degli attributi categorici per original_df
    modes_original = calculate_mode(original_df, categorical_qis)
    print("\nModa degli attributi categorici per original_df:")
    print(modes_original)


Moda degli attributi categorici per anonymized_df:
gender           ANY-GENDER
city                    USA
age                   18-81
profession          ANY-JOB
education     ANY-EDUCATION
Name: 0, dtype: object

Moda degli attributi categorici per original_df:
gender                        female
city                          Austin
age                             27.0
profession    Comic Book Illustrator
education                High School
Name: 0, dtype: object


Education level average e standard deviation:  

In [5]:
import json
import pandas as pd

# Carica il file JSON
with open('generation\\json\\educations.json', 'r') as f:
    education_levels = json.load(f)

# Funzione per creare la mappatura dei livelli di istruzione
def create_mapping(levels, start_value=1):  # Partiamo da 1, escludendo "ANY-EDUCATION"
    mapping = {}
    def recursive_map(levels, current_value):
        for level in levels:
            # Se il livello ha categorie, mappiamo i sotto-livelli allo stesso valore del livello principale
            if 'categories' in level and level['categories']:
                mapping[level['name']] = current_value
                for sub_level in level['categories']:
                    mapping[sub_level['name']] = current_value
                current_value += 1
            else:
                mapping[level['name']] = current_value
                current_value += 1
        return current_value

    recursive_map(levels['categories'], start_value)
    return mapping

# Crea la mappatura
education_mapping = create_mapping(education_levels)

# Stampa la mappatura
print("Mappatura dei livelli di istruzione:")
for key, value in education_mapping.items():
    print(f"{key}: {value}")

# Carica il dataset
df = pd.read_csv('generation\\database.csv')

# Verifica che la colonna esista nel dataset
if 'education' not in df.columns:
    raise KeyError("'education' column not found in the dataset")

# Sostituisci i valori della colonna "education" con i numeri interi corrispondenti
df['education'] = df['education'].map(education_mapping)

# Escludi i valori di "ANY-EDUCATION" (che non sono stati mappati)
df_filtered = df[df['education'].notna()]

# Calcola la media e la varianza della colonna codificata senza "ANY-EDUCATION"
mean_education = df_filtered['education'].mean()
var_education = df_filtered['education'].var()

print(f"\nMedia del livello di istruzione: {mean_education}")
print(f"Varianza del livello di istruzione: {var_education}")

# Salva il dataset aggiornato
# df.to_csv('dataset_updated.csv', index=False)


Mappatura dei livelli di istruzione:
High School: 1
Bachelor's Degree: 2
Graduate School: 3
Master's Degree: 3
Doctoral Degree: 3

Media del livello di istruzione: 1.749
Varianza del livello di istruzione: 0.7247237237237237


Gender average e standard deviation:  

In [6]:
import pandas as pd

# Crea il dizionario di mappatura
gender_mapping = {
    "male": 0,
    "female": 1
}

# Carica il dataset
df = pd.read_csv('generation/database.csv')

# Verifica che la colonna esista nel dataset
if 'gender' not in df.columns:
    raise KeyError("'gender' column not found in the dataset")

# Sostituisci i valori della colonna "gender" con i numeri interi corrispondenti
df['gender'] = df['gender'].map(gender_mapping)

# Calcola la media e la varianza della colonna "gender"
mean_gender = df['gender'].mean()
var_gender = df['gender'].var()

print(f"Media del genere: {mean_gender}")
print(f"Varianza del genere: {var_gender}")


Media del genere: 0.517
Varianza del genere: 0.249960960960961


In [7]:
# Mapping 

def mapping(col):
    if col == 'education':
        return education_mapping
    elif col == 'gender':
        return gender_mapping

Complete Statistical Analysis:

In [8]:
def statistical_analysis(original_df, anonymized_df, numerical_qis, categorical_qis, ordinal_qis):
    """ Esegue un'analisi statistica comparativa tra il dataset originale e quello anonimizzato """
   
    # Filtra i dataset per considerare solo le colonne di interesse
    original_df = original_df[all_qis]
    anonymized_df = anonymized_df[all_qis]

    # Analisi delle colonne numeriche
    print("Analisi delle colonne numeriche:")
    for col in numerical_qis:
        anonymized_df[col] = anonymized_df[col].apply(calculate_mean)
        
        mean_original = original_df[col].mean()
        mean_anonymized = anonymized_df[col].mean()
        std_original = original_df[col].std()
        std_anonymized = anonymized_df[col].std()

        print(f"Colonna: {col}")
        print(f"  Media originale: {mean_original}")
        print(f"  Media anonimizzata: {mean_anonymized}")
        print(f"  Deviazione standard originale: {std_original}")
        print(f"  Deviazione standard anonimizzata: {std_anonymized}")
        
    # Analisi colonne categoriche ordinali
    print("Analisi delle colonne categoriche ordinali:")
    for col in ordinal_qis:
        col_mapping = mapping(col)
        anonymized_df[col] = anonymized_df[col].map(col_mapping)
        original_df[col] = original_df[col].map(col_mapping)
        
        anonymized_filtered = anonymized_df[anonymized_df[col].notna()]
        original_filtered = original_df[original_df[col].notna()]
        
        mean_anonymized = anonymized_filtered[col].mean()
        std_anonymized = anonymized_filtered[col].var()
        mean_original = original_filtered[col].mean()
        std_original = original_filtered[col].var()
        
        print(f"Colonna: {col}")
        print(f"  Media originale: {mean_original}")
        print(f"  Media anonimizzata: {mean_anonymized}")
        print(f"  Deviazione standard originale: {std_original}")
        print(f"  Deviazione standard anonimizzata: {std_anonymized}")
        

    # Analisi delle colonne categoriche
    print("Analisi delle colonne categoriche:")
    modes_original = calculate_mode(original_df, categorical_qis)
    modes_anonymized = calculate_mode(anonymized_df, categorical_qis)
    
    for col in categorical_qis:
        print(f"Colonna: {col}")
        print(f"  Moda originale: {modes_original[col]}")
        print(f"  Moda anonimizzata: {modes_anonymized[col]}")


In [9]:
statistical_analysis(original_df, anonymized_df, numerical_qis, categorical_qis, ordinal_qis)


Analisi delle colonne numeriche:
Colonna: age
  Media originale: 52.984
  Media anonimizzata: 53.003
  Deviazione standard originale: 20.807996460897353
  Deviazione standard anonimizzata: 11.437648832291137
Analisi delle colonne categoriche ordinali:
Colonna: education
  Media originale: 1.749
  Media anonimizzata: 1.1851851851851851
  Deviazione standard originale: 0.7247237237237237
  Deviazione standard anonimizzata: 0.2636300897170462
Colonna: gender
  Media originale: 0.517
  Media anonimizzata: 0.5172413793103449
  Deviazione standard originale: 0.249960960960961
  Deviazione standard anonimizzata: 0.25066312997347473
Analisi delle colonne categoriche:
Colonna: gender
  Moda originale: 1.0
  Moda anonimizzata: 1.0
Colonna: city
  Moda originale: Austin
  Moda anonimizzata: USA
Colonna: age
  Moda originale: 27.0
  Moda anonimizzata: 52.0
Colonna: profession
  Moda originale: Comic Book Illustrator
  Moda anonimizzata: ANY-JOB
Colonna: education
  Moda originale: 1.0
  Moda anoni

C:\Users\aleba\AppData\Local\Temp\ipykernel_9984\2799938261.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anonymized_df[col] = anonymized_df[col].apply(calculate_mean)
C:\Users\aleba\AppData\Local\Temp\ipykernel_9984\2799938261.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anonymized_df[col] = anonymized_df[col].map(col_mapping)
C:\Users\aleba\AppData\Local\Temp\ipykernel_9984\2799938261.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [10]:
def discernability_penalty(df, qi):
    # Raggruppa il DataFrame per le colonne quasi-identificative
    partitions = df.groupby(qi).size()
    
    # Calcola la penalità come la somma dei quadrati delle dimensioni dei gruppi
    dp = sum(size**2 for size in partitions)
    
    return dp

In [11]:
discernability_penalty(anonymized_df, quasi_identifiers)

3148